In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    # 検索ツールの定義
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    # グラフのインスタンスを作成
    graph_builder = StateGraph(State)

    # 言語モデルの定義（ツールを紐づけ）
    llm = ChatOpenAI(model_name=model_name)
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノード
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    graph_builder.add_node("chatbot", chatbot)

    # ツールノード
    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)

    # 条件付きエッジ：ツール呼び出しが必要なら tools へ
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )

    # ツール実行後は chatbot に戻す
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノード
    graph_builder.set_entry_point("chatbot")

    # 記憶（スレッド）を持つグラフとしてコンパイル
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)
    return graph

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},  # 同じIDで会話の記憶を保持
        stream_mode="values",
    )

    # ツール出力（tool message）は表示せず、最終的なAI応答だけ表示
    for event in events:
        last = event["messages"][-1]
        if getattr(last, "type", "") == "ai" and getattr(last, "content", ""):
            print(last.content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# モデル名
MODEL_NAME = "gpt-4o-mini"

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("メッセージを入力:")
    if user_input.strip() == "":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)


こんにちは！今日はどんなことをお手伝いできますか？
1たす2は3です。何か他に知りたいことがありますか？
台湾観光に関する情報をいくつか見つけました。以下にまとめます。

1. **台湾の現地情報ブログ**（[Hankyu Travel](https://www.hankyu-travel.com/guide/taiwan/)）
   - 台湾旅行で楽しめる人気ドリンクスタンドや高雄の美食、プールのある「H2O水京棧國際酒店」などの情報があります。
   - 特に九份や台北、台中、台南、高雄などの観光地に関するガイドが充実しています。

2. **九份観景台**（[Hankyu Travel](https://www.hankyu-travel.com/guide/taiwan/)）
   - 九份は「千と千尋の神隠し」のモデルとも言われる観光地で、展望台からは九份と基隆嶼の美しいパノラマビューが楽しめます。
   - 日本統治時代の歴史を感じさせる場所で、カフェや雑貨店も点在し、散策が楽しいスポットです。

3. **台湾観光の研究**（[JTB](https://www.jtb.or.jp/wp-content/uploads/2016/01/bunka228-06.pdf)）
   - 台湾の観光産業に関する研究が進められており、観光客の増加や文化創意、食、夜市などが重要なテーマとして挙げられています。

台湾には美しい自然や歴史文化、現代的な楽しみが融合した観光地がたくさんあります。興味のある場所について、さらに詳しい情報をお探しすることもできますので、教えてください！
ありがとうございました!
